<left>
    <a href=img><img src = "https://github.com/tiagottmoraes/CS-s-Data-Science-Test/blob/main/Misc/1c86bfb3-cab0-4255-99bf-24c5488d888a-1632765661247.png?raw=true" width="400"  />
</left>

###### Developed by Tiago Moraes (09.02.2022)

# Data Science Test - Intro

----

The following notebooks presents the complete dataset analysis, exploration and pre-processing steps required to implement a tailored machine learning model designed to **predict the total minutes** ('total_minutes' label) it takes to a shopper to complete a given order.The rows not containing a total_minutes value were set aside to be used in the predictions phase.

For that, **3 different notebooks** were prepared for each task, as following:

- **[1. ETL process notebook:](#https://github.com/tiagottmoraes/CS-s-Data-Science-Test/blob/main/ETL.ipynb)** Complete ETL (data wrangling) process of the provided datasets, including feature engineering, feature transformations, and data standardizations, necessary to carry out all the statistics tests necessaries in the next phase;

- **[2. EDA notebook:](#https://github.com/tiagottmoraes/CS-s-Data-Science-Test/blob/main/ETL.ipynb)** Statistical exploration and understanding of the pre-processed dataset. Needed to define the best machine learning strategy for the target Label;

- **[3. Machine Learning Model development notebook:](#https://github.com/tiagottmoraes/CS-s-Data-Science-Test/blob/main/ETL.ipynb)** Build, train and test a ML model that takes in the process the dataset, generates a model and deploy it with unseen data to generate predictions ("total_minutes" label)
<br />
<br />

For this test goal, 4 different datasets were provided, each of them with different types of features and volume of data. 

**order_products.csv (198500 rows)**

*order_id*: ID of the order<br />
*product_id*: ID of the product<br />
*quantity*: The quantity ordered of this product<br />
*buy_unit*: The unit of the product (KG/UN)

**orders.csv (10000 rows)**

*order_id*: ID of the order<br />
*lat*: The latitude of the delivery location<br />
*lng*: The longitude of the delivery location<br />
*promised_time*: The delivery time promised to the user<br />
*on_demand*: If true, the order was promised to be delivered in less than X minutes<br />
*shopper_id*: ID representing the shopper completed the order<br />
*store_branch_id*: ID of the store branch<br />
***total_minutes***: The total minutes it took to complete the order (label)

**shopper.csv (2864 rows)**

*shopper_id*: ID of the shopper<br />
*seniority*: The experience level of the shopper<br />
*found_rate*: Percentage of products found by shopper historical<br />
*picking_speed*: Historical picking speed, products pr minutes<br />
*accepted_rate*: Percentage of orders historically accepted by shopper<br />
*rating*: client rating of shopper

**storebranch.csv (476 rows)**

*store_branch_id*: ID of the store branch<br />
*store*: ID representing the store<br />
*lat*: Latitude of the branch location<br />
*lng*: Longitude of the branch location
<br />
<br />


# 1. ETL process - Data wrangling, featute engineering and data standardization


----

## 1. Importing necessary modules

In [418]:
import pandas as pd
import numpy as np
from geopy import Point
from geopy.distance import distance

#### 1.1 Understanding primary keys between tables

Before start loading the datasets, let's take a look at how the tables are connected to each other and have a better undestanding of case at hand.



<center>
    <a href=img><img src = "https://github.com/tiagottmoraes/CS-s-Data-Science-Test/blob/main/entity_mapping.PNG?raw=true" width="800"  />
</center>

As we can see from the entity map above, primary keys from "storebranch.csv", "order_product.csv" , "shopper_ID.csv" can be found in the "orders.csv", which happens to be the table where the label ("total_minutes") is located. In this case, the better approach would be to unify (merge) all tables together, using the primary keys, into the table "order.csv". 

## 2. Importing datasets

### 2.1 Loading dataset "order_prod.csv"

After loading the order_prod.csv into a dataframe called **df_ordprod**, let's look at the first five rows using the `head()` function:

In [419]:
## Order_products
filename_ordprod = "https://raw.githubusercontent.com/tiagottmoraes/CS-s-Data-Science-Test/main/data/order_products.csv"
df_ordprod = pd.read_csv(filename_ordprod)
df_ordprod.head()


,order_id,product_id,quantity,buy_unit
0,47099653730fb1b76537fc10ad876255,c1244453d731c77416cb4766e3bd76cb,1.0,UN
1,689d8866915acf87e851c2591a23a82f,43cc2b100bec640fe563cd16f2db669f,1.0,KG
2,f26d16bf6f38c9e31d0be877f4013a9e,b8f880759d014134e272d881d49989a2,1.0,UN
3,161ccc896835ab41761b0e726becb6b1,dbc062b9bef805d27a6f4bea7edfe1f1,1.0,UN
4,4713deca10bb5db98fae150b52d61fc0,93a060f269bb569398921100f84c519a,2.0,UN


Checking missing data in df_ordprod:

In [420]:
missing_data_order = df_ordprod.isnull()
for column in missing_data_order.columns.values.tolist():
    print(column)
    print(missing_data_order[column].value_counts())
    print("")

order_id
False    198500
Name: order_id, dtype: int64

product_id
False    198500
Name: product_id, dtype: int64

quantity
False    198500
Name: quantity, dtype: int64

buy_unit
False    198500
Name: buy_unit, dtype: int64



NOTE:No missing values were found within this dataframe 

2.1.1 Feature engineering

In [421]:
print('Unique products listed:', df_ordprod['product_id'].nunique(),'\n'
      'Unique orders listed:', df_ordprod['order_id'].nunique())

Unique products listed: 15422 
Unique orders listed: 9978


The number of products is much bigger than the number of orders made, which is to be expected, since a single order takes many products. With that prospect in mind, an assumption made from now on is that, **what defines the complexity of a given order is the number of unique products, not the the quantity of each product**.<br />
<br />
A simple example that illustrates this: an order consisting of 12 different products (one item per product) demands much more attention and movement (and most likely *time*) from a shopper than what an order consisting of only 3 products, but with several items per product would.<br />
<br />
With that in mind, the "df_ordprod" was _grouped by the order_ID feature_, since that will give us the number of unique products present in each order, hence, giving us the indicator that is most likely linked to the complexity of a given order (which, in turn, makes orders take more time to be completed).

In [422]:
df_ordprod = df_ordprod.groupby(["order_id"], as_index=False).count()
df_ordprod.rename(columns={'product_id':'order_size'}, inplace=True)
df_ordprod.head()

,order_id,order_size,quantity,buy_unit
0,0004a3841c1eeb6c6e77585a941c21e0,4,4,4
1,0005a6ecbbde1e8d273f5577bcff2c9c,1,1,1
2,0007baeb6700fc203be2d1f1e11222d7,22,22,22
3,0012195a6a8ca9ec308a3010eeea8ebc,11,11,11
4,0013011fa72b498b9feb84f4e7104980,44,44,44


NOTE: "product_id" was renamed to "order_size", since it better represents the number of unique products (list of items) contained in each order

### 2.2 Loading dataset "shopper.csv"

After loading the shoppers.csv into a dataframe called **df_shop**, let's look at the first five rows using the `head()` function:

In [423]:
## Shopper
filename_shop = "https://raw.githubusercontent.com/tiagottmoraes/CS-s-Data-Science-Test/main/data/shoppers.csv"
df_shop = pd.read_csv(filename_shop)
df_shop.head()

,shopper_id,seniority,found_rate,picking_speed,accepted_rate,rating
0,1fc20b0bdf697ac13dd6a15cbd2fe60a,41dc7c9e385c4d2b6c1f7836973951bf,0.8606,1.94,1.00,4.87
1,e1c679ac73a69c01981fdd3c5ab8beda,6c90661e6d2c7579f5ce337c3391dbb9,0.8446,1.23,0.92,4.92
2,09d369c66ca86ebeffacb133410c5ee1,6c90661e6d2c7579f5ce337c3391dbb9,0.8559,1.56,1.00,4.88
3,db39866e62b95bb04ebb1e470f2d1347,50e13ee63f086c2fe84229348bc91b5b,NaN,2.41,NaN,NaN
4,8efbc238660053b19f00ca431144fdae,6c90661e6d2c7579f5ce337c3391dbb9,0.8770,1.31,0.92,4.88


Checking missing data in df_shop:

In [424]:
missing_data_order = df_shop.isnull()
for column in missing_data_order.columns.values.tolist():
    print(column)
    print(missing_data_order[column].value_counts())
    print("")

shopper_id
False    2864
Name: shopper_id, dtype: int64

seniority
False    2864
Name: seniority, dtype: int64

found_rate
False    2763
True      101
Name: found_rate, dtype: int64

picking_speed
False    2864
Name: picking_speed, dtype: int64

accepted_rate
False    2837
True       27
Name: accepted_rate, dtype: int64

rating
False    2780
True       84
Name: rating, dtype: int64



NOTE: There were found:
- 101 NaN elements in the 'found_rate' feature
- 27 NaN elements in the 'accepted_rate' feature
- 84 NaN elements in the 'rating' feature

In [425]:
print('Unique shoppers listed:', df_shop['shopper_id'].nunique(),'\n' 
      'Unique seniority listed:', df_shop['seniority'].nunique())

Unique shoppers listed: 2864 
Unique seniority listed: 4


Despite being anonymized, the "seniority" feature consists of only 4 different classes. In order to help clarify this feature and its relationship with other elements in this dataframe, the original hash code was replaced by random letters (T, I, M, W) and had its data type replaced by categorical.

In [426]:
df_shop['seniority'].unique()

array(['41dc7c9e385c4d2b6c1f7836973951bf',
       '6c90661e6d2c7579f5ce337c3391dbb9',
       '50e13ee63f086c2fe84229348bc91b5b',
       'bb29b8d0d196b5db5a5350e5e3ae2b1f'], dtype=object)

In [427]:
df_shop.replace('41dc7c9e385c4d2b6c1f7836973951bf', 'T', inplace=True)
df_shop.replace('6c90661e6d2c7579f5ce337c3391dbb9','I', inplace=True)
df_shop.replace('50e13ee63f086c2fe84229348bc91b5b','M', inplace=True)
df_shop.replace('bb29b8d0d196b5db5a5350e5e3ae2b1f','W', inplace=True)
df_shop ['seniority']= df_shop['seniority'].astype('category')

In [428]:
# Counting the number of shoppers in each seniority class
df_shop['seniority'].value_counts()

I    1643
M     719
T     440
W      62
Name: seniority, dtype: int64

In [429]:
df_shop.sample(2)

,shopper_id,seniority,found_rate,picking_speed,accepted_rate,rating
1122,8039278c212460c9af5a3dab089fda26,I,0.8493,1.67,1.00,4.88
2670,07b121dbae684d1b2b824a3320872874,I,0.8787,1.19,0.88,4.92


### 2.3 Loading dataset "storebranch.csv"

After loading the storebranch.csv into a dataframe called **df_store**, let's look at the first five rows using the `head()` function:

In [430]:
## Storebranch
filename_store = "https://raw.githubusercontent.com/tiagottmoraes/CS-s-Data-Science-Test/main/data/storebranch.csv"
df_store = pd.read_csv(filename_store)
df_store.head()

,store_branch_id,store_id,lat,lng
0,aff1621254f7c1be92f64550478c56e6,92cc227532d17e56e07902b254dfad10,-33.422497,-70.609231
1,56352739f59643540a3a6e16985f62c7,0336dcbab05b9d5ad24f4333c7658a0e,-33.385484,-70.555579
2,7d04bbbe5494ae9d2f5a76aa1c00fa2f,9bf31c7ff062936a96d3c8bd1f8f2ff3,-33.416579,-70.565224
3,2b24d495052a8ce66358eb576b8912c8,c4ca4238a0b923820dcc509a6f75849b,-33.512578,-70.655952
4,5487315b1286f907165907aa8fc96619,d82c8d1619ad8176d665453cfb2e55f0,-33.347645,-70.542229


Checking missing data in df_store:

In [431]:
missing_data_order = df_store.isnull()
for column in missing_data_order.columns.values.tolist():
    print(column)
    print(missing_data_order[column].value_counts())
    print("")

store_branch_id
False    476
Name: store_branch_id, dtype: int64

store_id
False    476
Name: store_id, dtype: int64

lat
False    476
Name: lat, dtype: int64

lng
False    476
Name: lng, dtype: int64



NOTE:No missing values were found within this dataframe 

In [432]:
print('Unique store_branch listed:', df_store['store_branch_id'].nunique(),'\n' 'Unique store listed:', df_store['store_id'].nunique())

Unique store_branch listed: 476 
Unique store listed: 221


Considering that both "storebranch.csv" and "orders.csv" have different data on latitude and longitude coordinates but uses the same name convention, let's change both column's names to help differentiate them

In [433]:
# renaming the columns
df_store = df_store.rename(columns={'lat':'store_lat', 'lng':'store_long'})

### 2.4 Loading dataset "orders.csv"

After loading the orders.csv into a dataframe called **df_order**, let's look at the first five rows using the `head()` function:

In [434]:
## ORDERS - LABEL
filename_ORDER = "https://raw.githubusercontent.com/tiagottmoraes/CS-s-Data-Science-Test/main/data/orders.csv"
df_ORDER = pd.read_csv(filename_ORDER)
df_ORDER.head()

,order_id,lat,lng,promised_time,on_demand,shopper_id,store_branch_id,total_minutes
0,e750294655c2c7c34d83cc3181c09de4,-33.501675,-70.579369,2019-10-18 20:48:00+00:00,True,e63bc83a1a952fa2b3cc9d558fb943cf,65ded5353c5ee48d0b7d48c591b8f430,67.684264
1,6581174846221cb6c467348e87f57641,-33.440584,-70.556283,2019-10-19 01:00:00+00:00,False,195f9e9d84a4ba9033c4b6a756334d8b,45fbc6d3e05ebd93369ce542e8f2322d,57.060632
2,3a226ea48debc0a7ae9950d5540f2f34,-32.987022,-71.544842,2019-10-19 14:54:00+00:00,True,a5b9ddc0d82e61582fca19ad43dbaacb,07563a3fe3bbe7e3ba84431ad9d055af,NaN
3,7d2ed03fe4966083e74b12694b1669d8,-33.328075,-70.512659,2019-10-18 21:47:00+00:00,True,d0b3f6bf7e249e5ebb8d3129341773a2,f1748d6b0fd9d439f71450117eba2725,52.067742
4,b4b2682d77118155fe4716300ccf7f39,-33.403239,-70.564020,2019-10-19 20:00:00+00:00,False,5c5199ce02f7b77caa9c2590a39ad27d,1f0e3dad99908345f7439f8ffabdffc4,140.724822


Checking missing data in df_ORDER:

In [435]:
missing_data_order = df_ORDER.isnull()
for column in missing_data_order.columns.values.tolist():
    print(column)
    print(missing_data_order[column].value_counts())
    print("")

order_id
False    10000
Name: order_id, dtype: int64

lat
False    10000
Name: lat, dtype: int64

lng
False    10000
Name: lng, dtype: int64

promised_time
False    10000
Name: promised_time, dtype: int64

on_demand
False    10000
Name: on_demand, dtype: int64

shopper_id
False    10000
Name: shopper_id, dtype: int64

store_branch_id
False    10000
Name: store_branch_id, dtype: int64

total_minutes
False    8000
True     2000
Name: total_minutes, dtype: int64



NOTE: No null values, EXCEPT for the label "total_minutes". The rows containg this missing values in the label will be later taken apart to be used during the predictions phase.

Considering that both "storebranch.csv" and "orders.csv" have different data on latitude and longitude coordinates but uses the same name convention, let's change both column's names to help differentiate them.

In [436]:
df_ORDER = df_ORDER.rename(columns={'lat':'delivery_lat', 'lng':'delivery_long'})

### 2.5 Merging dataframes

Merging dataframes using primary keys

1. First, merge df_ordprod in df_ORDER, using "order_id" as primary keys:

In [437]:
df_merge = pd.merge(df_ORDER, df_ordprod, on='order_id')

2. Then, merge df_shop in df_merge (result from previous merge operation), using "shopper_id" as foreing keys:

In [438]:
df_merge = pd.merge(df_merge, df_shop, on='shopper_id')

3. Finally, merge df_store in df_merge (result from previous merge operation), using "store_branch_id" as foreing keys:

In [439]:
df_merge = pd.merge(df_merge, df_store, on='store_branch_id')

After merging and, then, sorting the features, we get the following dataframe:

In [440]:
df_merge = df_merge.reindex(['order_id','store_branch_id','store_lat','delivery_lat','store_long','delivery_long','promised_time', 'store_id','quantity', 'buy_unit','shopper_id','on_demand','seniority', 'order_size', 'found_rate','picking_speed','accepted_rate','rating', 'total_minutes'], axis=1)
pd.set_option('display.max_columns', None)
df_merge.sample(2)

,order_id,store_branch_id,store_lat,delivery_lat,store_long,delivery_long,promised_time,store_id,quantity,buy_unit,shopper_id,on_demand,seniority,order_size,found_rate,picking_speed,accepted_rate,rating,total_minutes
7829,667708f477fad2705f50071023bac4ce,26657d5ff9020d2abefe558796b99584,-33.355373,-33.350095,-70.538246,-70.488018,2019-10-19 15:00:00+00:00,6f4922f45568161a8cdf4ad2299f6d23,3,3,c303814b79782cab2b23387895adf683,False,I,3,0.8474,1.56,0.96,4.68,NaN
1134,3c59a6cbe7c2d84889e823e05879cc7d,1679091c5a880faf6fb5e6087eb1b2dc,-33.370765,-33.385671,-70.512420,-70.503318,2019-10-19 16:00:00+00:00,c4ca4238a0b923820dcc509a6f75849b,78,78,e9ae346da1cabad6b408487c7d1e09cf,False,I,78,0.8885,1.74,0.88,4.84,109.432638


## 3 Dealing with missing data

### 3.1 Removing/Replacing missing values

First, let's remove some unnecessary features, such as "promissed_time" (nothing to compare with), "store_id" (already used store_branch_id as primary key),"quantity" and "buy_unit" (both deprecated after grouping by "order_id", as show in item 2.1)

In [441]:
df_merge.drop(['promised_time', 'store_id','quantity', 'buy_unit'], axis=1, inplace=True)

As shown before, some features presents missing values that must be addressed before we head to the exploratory analysis. 


In [442]:
df_merge.sample(2)

,order_id,store_branch_id,store_lat,delivery_lat,store_long,delivery_long,shopper_id,on_demand,seniority,order_size,found_rate,picking_speed,accepted_rate,rating,total_minutes
5545,55482d7e54fc0fda8be73e7db166a545,5751ec3e9a4feab575962e78e006250d,-33.463001,-33.456859,-70.575301,-70.572237,e8eaa7ab90d6ffe2474100103b723e96,True,M,6,0.8098,3.74,0.92,4.72,48.170608
36,c4531b795fc4c119ae5517fd62f86e8c,65ded5353c5ee48d0b7d48c591b8f430,-33.485280,-33.495454,-70.579250,-70.554772,6bcd77f7e04386ca53129137ad0c35e1,False,I,56,0.8903,1.29,0.96,4.92,139.862937


In [443]:
#checking the overall distribution and volume of the dataframe
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9978 entries, 0 to 9977
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   order_id         9978 non-null   object  
 1   store_branch_id  9978 non-null   object  
 2   store_lat        9978 non-null   float64 
 3   delivery_lat     9978 non-null   float64 
 4   store_long       9978 non-null   float64 
 5   delivery_long    9978 non-null   float64 
 6   shopper_id       9978 non-null   object  
 7   on_demand        9978 non-null   bool    
 8   seniority        9978 non-null   category
 9   order_size       9978 non-null   int64   
 10  found_rate       9779 non-null   float64 
 11  picking_speed    9978 non-null   float64 
 12  accepted_rate    9932 non-null   float64 
 13  rating           9816 non-null   float64 
 14  total_minutes    7983 non-null   float64 
dtypes: bool(1), category(1), float64(9), int64(1), object(3)
memory usage: 1.1+ MB


In [444]:
#checking missing values in merged dataframe
print(" \nTotal number of NaN items present in the columns of the DataFrame 'df_merge': \n\n", df_merge.isnull().sum())

 
Total number of NaN items present in the columns of the DataFrame 'df_merge': 

 order_id              0
store_branch_id       0
store_lat             0
delivery_lat          0
store_long            0
delivery_long         0
shopper_id            0
on_demand             0
seniority             0
order_size            0
found_rate          199
picking_speed         0
accepted_rate        46
rating              162
total_minutes      1995
dtype: int64


As stated before, the "total_minutes" label will be set apart to be used in the preditions phase, so it won't have any new values imputed. For the rest of features ('found_rate', 'accepted_rate' and 'rating '), all missing values will be imputed by using their respectives average values in the series.

In [445]:
# calculating the averages
avg_found = df_merge['found_rate'].mean(axis=0)
avg_accept = df_merge['accepted_rate'].mean(axis=0)
avg_rate = df_merge['rating'].mean(axis=0)
print("Average of found_rate:", avg_found )
print("Average of accepted_rate:", avg_accept )
print("Average of rating:", avg_rate )


Average of found_rate: 0.8633177727784027
Average of accepted_rate: 0.9170133784336325
Average of rating: 4.849341890790547


In [446]:
# replacing the NaN with the average values
df_merge['found_rate'].replace(np.nan, avg_found, inplace=True)
df_merge['accepted_rate'].replace(np.nan, avg_accept, inplace=True)
df_merge['rating'].replace(np.nan, avg_rate, inplace=True)

In [447]:
print(" \nTotal number of NaN items present in the columns of the DataFrame 'df_merge': \n\n", df_merge.isnull().sum())

 
Total number of NaN items present in the columns of the DataFrame 'df_merge': 

 order_id              0
store_branch_id       0
store_lat             0
delivery_lat          0
store_long            0
delivery_long         0
shopper_id            0
on_demand             0
seniority             0
order_size            0
found_rate            0
picking_speed         0
accepted_rate         0
rating                0
total_minutes      1995
dtype: int64


NOTE: All missing values removed from the desired the features

### 3.2 Feature transformation

Geopy library can calculate straight distances between two points using only their geographic coordinates. First, let's change the latitude and longitude values and convert it into a single geographic point using Geopy's `Point` function.

In [448]:
df_merge['store_coord'] = df_merge.apply(lambda row: Point(latitude=row['store_lat'], longitude=row['store_long']), axis=1)
df_merge['delivery_coord'] = df_merge.apply(lambda row: Point(latitude=row['delivery_lat'], longitude=row['delivery_long']), axis=1)
df_merge = df_merge.drop(['store_lat', 'store_long', 'delivery_lat', 'delivery_long'], axis=1) # dropping the unnecessary columns
df_merge.sample(1)

,order_id,store_branch_id,shopper_id,on_demand,seniority,order_size,found_rate,picking_speed,accepted_rate,rating,total_minutes,store_coord,delivery_coord
1032,6239971cc9b841f883e8ee644b32d191,1679091c5a880faf6fb5e6087eb1b2dc,7364b2cb7b2acdaa8e02cfaf26218ec5,True,T,7,0.8935,2.62,0.96,4.85,58.753143,"33 22m 14.754s S, 70 30m 44.712s W","33 19m 26.4429s S, 70 32m 8.02231s W"


Finally, using Geopy's `Distance` function, we can obtain the straight line distance (km) between two given points.

In [449]:
#calculating the distance between the store and the delivery point, using their coordinates
df_merge['distance_km'] = df_merge.apply(lambda row: distance(row['delivery_coord'], row['store_coord']).km , axis=1)

In [450]:
#removing coordinate data and renaming the newly found distance
df_merge.drop(['delivery_coord', 'store_coord'], axis=1, inplace=True)
df_merge = df_merge.reindex(['order_id','store_branch_id','shopper_id','on_demand','seniority', 'order_size', 'found_rate','picking_speed','accepted_rate','rating','distance_km','total_minutes'], axis=1)
df_merge.sample(2)

,order_id,store_branch_id,shopper_id,on_demand,seniority,order_size,found_rate,picking_speed,accepted_rate,rating,distance_km,total_minutes
4484,608018e48a2b836dd1ee1b79e8fb6bdd,c4ca4238a0b923820dcc509a6f75849b,1de1ce4ee8f938c6fe9d6dfe95c9804f,True,I,29,0.8959,1.13,1.00,4.80,1.825195,67.299601
2053,42a0278f2f61e167d8efe9c61ad4206d,1679091c5a880faf6fb5e6087eb1b2dc,304ed3a3e09c21867a8bc2386f635389,False,I,32,0.9496,1.00,0.88,4.92,0.211524,79.713877


One last transformation to be carried out is the "on_demand" feature, which is currently a boolean. Since booleans aren't good matchs when when working with machine learning models, we'll simply replace "True"=1 and "False"=0

In [451]:
df_merge['on_demand'].dtypes

dtype('bool')

In [452]:
# True = 1, False = 0
df_merge['on_demand'].replace(True,1, inplace=True)
df_merge['on_demand'].replace(False,0, inplace=True)

In [453]:
df_merge.sample(4)

,order_id,store_branch_id,shopper_id,on_demand,seniority,order_size,found_rate,picking_speed,accepted_rate,rating,distance_km,total_minutes
226,93d30249cf385540ec47df8624062684,45fbc6d3e05ebd93369ce542e8f2322d,76b112f3a9a16d2120ea5e842c3352b6,1,M,22,0.8038,3.05,0.840,5.00,2.638877,NaN
6216,b02e19d80831f2e22aa48c56d9a337a8,7ef605fc8dba5425d6965fbd4c8fbe1f,c0354e7babed01459c603b9dd8daba79,1,T,1,0.8472,2.18,0.875,5.00,2.123429,45.935544
2120,c036f92a8edd9d057b57506459589473,1679091c5a880faf6fb5e6087eb1b2dc,c60f1209a02a5abf9e47688ff8119e2e,1,I,28,0.8863,1.06,1.000,4.96,2.154335,NaN
8534,aac988ef6aa8bb05a2166f35696bde8a,b056eb1587586b71e2da9acfe4fbd19e,1894e03459c66a8fe51a33b240520b55,1,M,2,0.8185,2.80,0.880,4.88,1.303593,37.484178


# 4 Exporting datasets

4.1 Exporting test dataset ("total_minutes" = NaN) using `pd.isna`

In [454]:
# pd.isna returns only a dataframe with the missing values on the "total_minutes" column
df_TEST = df_merge.loc[(pd.isna(df_merge['total_minutes']))]
df_TEST.to_csv('df_TEST.csv')

4.2 Exporting exploration/training dataset ("total_minutes" != NaN) using `pd.notna`

In [455]:
# pd.notna returns only a dataframe with the non-missing values on the "total_minutes" column
df_TRAIN = df_merge.loc[(pd.notna(df_merge['total_minutes']))]
df_TRAIN.to_csv('df_TRAIN.csv')

In [456]:
#final dataset before EDA and ML modeling
df_TRAIN.sample(5)

,order_id,store_branch_id,shopper_id,on_demand,seniority,order_size,found_rate,picking_speed,accepted_rate,rating,distance_km,total_minutes
4500,8d1c2c809c549c0993108e7efaaa76ea,c4ca4238a0b923820dcc509a6f75849b,9f675c7ea36e93639f4338ec5e9ea10e,1,T,8,0.8640,2.08,0.96,4.96,1.016127,78.285455
2693,28d9082977af9000730cb916f49d18e2,1f0e3dad99908345f7439f8ffabdffc4,c3e7fbc14afdf4073804fd25c8f2edce,0,I,45,0.8951,1.26,0.96,4.84,5.704638,120.978157
7499,b8c05693696e17c89bd09020b97d6605,9461cce28ebe3e76fb4b931c35a169b0,ef74a6e66d66a95a09609eb91c5da45a,1,T,25,0.8571,2.83,0.92,4.92,11.231542,136.385140
309,fee6b12c266491548d6caf8f724231ea,45fbc6d3e05ebd93369ce542e8f2322d,66f0c22f9672f68dcfe8789a5fb0cf42,0,M,17,0.8130,1.43,0.76,4.64,2.103943,52.900673
3296,b455294bf49ab97ee2a0c7be0c0f1576,1f0e3dad99908345f7439f8ffabdffc4,4d6aa322dc215e96b9bbb1e6ca3a6e1f,0,I,27,0.8895,1.20,0.96,4.92,2.424081,134.900776
